# About this notebook

In this notebook I extend the PurgedGroupTimeSeriesSplit to allow an additional set used to validate stacking or ensemble classifiers. 
The new class is called PurgedGroupTimeSeriesSplitStacking. When using stacking_mode=False it reverts to PurgedGroupTimeSeriesSplit behaviour.

##### Some clarification about terms

In a stacking classifier context I refer to:
- training set as the set where we fit the base classifiers
- validation set as the set where we evaluate base classifier and fit the stacking classifier/final estimator
- test/stacking set as the set where we evaluate the stacking classifier/final estimator predictions


### Notebooks Sections:

0. [*Load Competition Dataset*](#data_load)<br>

1. [*PurgedGroupTimeSeriesSplitStacking Class Definition*](#class_definition)<br>

2. [*PurgedGroupTimeSeriesSplitStacking example on Competition Data*](#class_example)



### Props to:

1 -  [GroupTimeSeriesSplit](https://www.kaggle.com/jorijnsmit/found-the-holy-grail-grouptimeseriessplit) 
    
2 - [PurgedGroupTimeSeriesSplit](https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna), extension of 1 allowing for gaps between end of training and start of testing

[Here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) a reference for StackingClassifiers.

In [ ]:
import numpy as np
import datatable as dt
import pandas as pd
import tqdm
import os
import re
import logging

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)
    
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [20, 12]  # width, height

import gc

input_path = '/kaggle/input/'
root_path = os.path.join(input_path, 'jane-street-market-prediction')

<a id ="data_load"></a>
### Load Competition Dataset 

In [ ]:
%%time

train = (dt.fread(os.path.join(root_path, "train.csv")).to_pandas()
        .query('weight > 0')#.pipe(reduce_mem_usage)
        .reset_index(drop = True))

train['action'] = (train.resp > 0).astype(int)

resp_cols = [i for i in train.columns if 'resp' in i]
meta_features = dt.fread(os.path.join(root_path, "features.csv")).to_pandas()

features_names = [i for i in train.columns if 'feature_' in i]
features_index = list(map(lambda x: int(re.sub("feature_", "", x)), features_names))
features_tuples = sorted(list(zip(features_names, features_index)), key = lambda x: x[1])
just_features = [i[0] for i in features_tuples]

In [ ]:
# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py

name_dict = {True: 'With_Stacking_Set', 
             False: 'No_Stacking_Set'}

def plot_cv_indices_stacking(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, indices_split in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        
        indices = np.array([np.nan] * len(X))
        indices[indices_split[0]] = 1
        indices[indices_split[1]] = 0
        if cv.stacking_mode:
            indices[indices_split[2]] = -1

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)
    
    if cv.stacking_mode:
        ax.scatter(range(len(X)), [ii + 3.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    
    ax.set_title('{}'.format(name_dict[cv.stacking_mode]), fontsize=15)
    #ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

<a id = "class_definition"></a>

### PurgedGroupTimeSeriesSplitStacking Class Definition

Here (hidden) I write the enhanced PurgedGroupTimeSeriesSplit class. 


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

class PurgedGroupTimeSeriesSplitStacking(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    stacking_mode : bool, default=True
        Whether to provide an additional set to test a stacking classifier or not. 
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    max_val_group_size : int, default=Inf
        Maximum group size for a single validation set.
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split, if stacking_mode = True and None 
        it defaults to max_val_group_size.
    val_group_gap : int, default=None
        Gap between train and validation
    test_group_gap : int, default=None
        Gap between validation and test, if stacking_mode = True and None 
        it defaults to val_group_gap.
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 stacking_mode=True,
                 max_train_group_size=np.inf,
                 max_val_group_size=np.inf,
                 max_test_group_size=np.inf,
                 val_group_gap=None,
                 test_group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.max_val_group_size = max_val_group_size
        self.max_test_group_size = max_test_group_size
        self.val_group_gap = val_group_gap
        self.test_group_gap = test_group_gap
        self.verbose = verbose
        self.stacking_mode = stacking_mode
        
    def split(self, X, y=None, groups=None):
        if self.stacking_mode:
            return self.split_ensemble(X, y, groups)
        else:
            return self.split_standard(X, y, groups)
        
    def split_standard(self, X, y=None, groups=None):
        """Generate indices to split data into training and validation set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/validation set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        val : ndarray
            The validation set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_splits = self.n_splits
        group_gap = self.val_group_gap
        max_val_group_size = self.max_val_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_val_size = min(n_groups // n_folds, max_val_group_size)
        group_val_starts = range(n_groups - n_splits * group_val_size,
                                  n_groups, group_val_size)
        for group_val_start in group_val_starts:
            train_array = []
            val_array = []

            group_st = max(0, group_val_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_val_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for val_group_idx in unique_groups[group_val_start:
                                                group_val_start +
                                                group_val_size]:
                val_array_tmp = group_dict[val_group_idx]
                val_array = np.sort(np.unique(
                                              np.concatenate((val_array,
                                                              val_array_tmp)),
                                     axis=None), axis=None)

            val_array  = val_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in val_array]
            
    def split_ensemble(self, X, y=None, groups=None):
        """Generate indices to split data into training, validation and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        val : ndarray
            The validation set indices for that split (testing indices for base classifiers).
        test : ndarray
            The testing set indices for that split (testing indices for final classifier)
        """

        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
            
        X, y, groups = indexable(X, y, groups)
        n_splits = self.n_splits
        val_group_gap = self.val_group_gap
        test_group_gap = self.test_group_gap
        if test_group_gap is None:
            test_group_gap = val_group_gap
        max_train_group_size = self.max_train_group_size
        max_val_group_size = self.max_val_group_size
        max_test_group_size = self.max_test_group_size
        if max_test_group_size is None:
            max_test_group_size = max_val_group_size
            
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)

        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_val_size = min(n_groups // n_folds, max_val_group_size)
        group_test_size = min(n_groups // n_folds, max_test_group_size)
        
        group_test_starts = range(n_groups - n_splits * group_test_size, n_groups, group_test_size)
        train_indices= []
        val_indices= []
        test_indices= []
        
        for group_test_start in group_test_starts:

            train_array = []
            val_array = []
            test_array = []
            
            val_group_st = max(max_train_group_size + val_group_gap, 
                               group_test_start - test_group_gap - max_val_group_size)

            train_group_st = max(0, val_group_st - val_group_gap - max_train_group_size)

            for train_group_idx in unique_groups[train_group_st:(val_group_st - val_group_gap)]:

                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size

            for val_group_idx in unique_groups[val_group_st:(group_test_start - test_group_gap)]:
                val_array_tmp = group_dict[val_group_idx]
                val_array = np.sort(np.unique(
                                              np.concatenate((val_array,
                                                              val_array_tmp)),
                                     axis=None), axis=None)

            val_array  = val_array[val_group_gap:]

            for test_group_idx in unique_groups[group_test_start:(group_test_start + group_test_size)]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[test_group_gap:]

            yield [int(i) for i in train_array], [int(i) for i in val_array], [int(i) for i in test_array]

<a id = "class_example"></a>

### PurgedGroupTimeSeriesSplitStacking example on Competition Data

Here I define two CrossValidation strategies:

- the first one has an additional stacking set

- the second one has no stacking set 

In [ ]:
cv = PurgedGroupTimeSeriesSplitStacking(
    n_splits=5,
    stacking_mode = True,
    max_train_group_size=15,
    max_test_group_size=5,
    max_val_group_size=5,
    val_group_gap=2,
    test_group_gap=1
)

cv_no_stacking = PurgedGroupTimeSeriesSplitStacking(
    n_splits=5,
    stacking_mode = False,
    max_train_group_size=15,
    max_val_group_size=5,
    val_group_gap=2
)

In [ ]:
fig, ax = plt.subplots(1, 1)

plot_cv_indices_stacking(cv, train.query('date < 50')[just_features], train.query('date < 50')['action'], train.query('date < 50')['date'], 
                         ax, 5, lw=20);

fig, ax = plt.subplots(1, 1)

plot_cv_indices_stacking(cv_no_stacking, train.query('date < 50')[just_features], train.query('date < 50')['action'], train.query('date < 50')['date'], 
                         ax, 5, lw=20);
